In [1]:
import pandas as pd
from os import getcwd, listdir
from os.path import join, isfile, exists, abspath, pardir
from sys import path
import re

##### Configs

In [2]:
database = "sdm"
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")

In [3]:
path.append(scripts_dir)

In [4]:
from connect import Neo4jConnection
import utils 

In [5]:
driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

In [6]:
q = """
MATCH c=(a:Author)<-[p:written_by]-(d:Document)-[r:published_in]->(j:Journal)
RETURN c limit 10
"""

In [8]:
utils.run_query_with_result(driver, q, raw=True)

TypeError: run_query_with_result() takes 1 positional argument but 2 were given

In [8]:
from neo4j import GraphDatabase

In [9]:
driver = GraphDatabase.driver("bolt://localhost:7687")

In [59]:
def get_two_tx(tx):
    result = tx.run("""
        Match (d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper' and toInteger(d.cited_count) > 0
        With j.name as conference, d.title as paper, sum(toInteger(d.cited_count)) as cited_count
        order by cited_count desc
        return conference, collect(paper)[..3] as papers, collect(cited_count)[..3] as total_cited_count
    """)
    values = list()
    for ix, record in enumerate(result):
        # if ix > 1:
        #     break
        value = record.values()
        if value:
            # if isinstance(value, list) and len(value):
            #     value = value[0]
            values.append(value)
    info = result.consume()  # discard the remaining records if there are any
    # use the info for logging etc.
    return values, info

In [60]:
with driver.session(database=database) as session:
    values, info = session.read_transaction(get_two_tx)

In [ ]:
pd.DataFrame(values)

In [85]:
values = list()
for ix, record in enumerate(run_query(q)):
    # if ix > 1:
    #     break
    value = record.values()
    if value:
        # if isinstance(value, list) and len(value):
        #     value = value[0]
        values.append(value)
# info = r.consume()

In [86]:
values

[]